In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import os

# Obtener ruta absoluta 
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

plt.style.use("seaborn-v0_8-whitegrid")
from src.data.clean_columns import clean_dataframe_columns
from src.utils.constants import(
    VARS_BINARIAS,
    VARS_CATEGORICAS_NOMINALES,
    VARS_CATEGORICAS_ORDINALES,
    VARS_NUMERICAS,
    TARGET,
    TARGET_VALUES,
    LABELS    
)

In [ ]:
# Cargar dataset

df_raw = pd.read_csv('../data/raw/data.csv', delimiter=';')
df = df_raw.copy()  

df = clean_dataframe_columns(df)

print("Dataset cargado correctamente\n")
df.head()

# 1. VARIABLES NUMERICAS

## 1.1. ESCALADO (SOLO APLICA PARA MODELO BASELINE RL)

In [ ]:
# =============================================================================
# PASO X: Escalado
# =============================================================================


######################################################################
#Para XGBoost/LightGBM/CatBoost: NO escalar
######################################################################



#from sklearn.preprocessing import StandardScaler

#from sklearn.preprocessing import StandardScaler

# Solo para el baseline (Logistic Regression)
# StandardScaler: media=0, std=1
#scaler = StandardScaler()
#X_train_scaled = scaler.fit_transform(X_train)
#X_test_scaled = scaler.transform(X_test)


# RobustScaler() en vez de StandardScaler()¿?


## 1.2. VARIABLES CON SESGO (SOLO APLICA PARA MODELO BASELINE RL)

In [ ]:
# Opción 1: Log transform (para sesgo positivo)
# VARIABLES: age_at_enrollment, curricular_units_1st_sem_credited, curricular_units_2nd_sem_credited, curricular_units_1st_sem_without_evaluations
# df['age_log'] = np.log1p(df['age_at_enrollment'])


# Opción 2: Square root transform (más suave)
# df['age_sqrt'] = np.sqrt(df['age_at_enrollment'])


## 1.3. VARIABLES ZERO-INFLATED (SOLO APLICA PARA MODELO BASELINE RL)

In [ ]:
# Opción 1: Crear variable binaria adicional
# VARIABLES: curricular_units_1st_sem_grade, curricular_units_2nd_sem_grade, curricular_units_1st_sem_without_evaluations, curricular_units_2nd_sem_without_evaluations.

#OPCIONES
#BINARIZACION
#df['has_credited'] = (df['curricular_units_1st_sem_credited'] > 0).astype(int)
# Transformación log(x + 1)
#df['credited_log'] = np.log1p(df['curricular_units_1st_sem_credited'])



# 2. VARIABLES CATEGORICAS

## 2.1. VARIABLES CATEGORICAS NOMINALES

### 2.1.1 Variable marital_status

In [ ]:
df['is_single'] = (df['marital_status'] == 1).astype(int) 

# Verificar print("Distribución de is_single:") 
print(df['is_single'].value_counts())
print(f"\nTasa deserción solteros (1): {df[df['is_single']==1]['target'].eq('Dropout').mean()*100:.1f}%") 
print(f"Tasa deserción no solteros (0): {df[df['is_single']==0]['target'].eq('Dropout').mean()*100:.1f}%") 

df['is_single'].hist()

La variable marital_status fue binarizada (Soltero vs No Soltero) debido a la presencia de categorías con muy pocos casos (Viudo: 4, Separado: 6) y al patrón claro observado: los estudiantes no solteros presentan una tasa de deserción del 47% frente al 30% de los solteros, probablemente debido a mayores responsabilidades familiares y laborales."

### 2.1.2 Variable aplication_mode

In [ ]:
# Se agrupa en 3 categoria por nivel de riesgo
def agrupar_application_mode(x):
    # ALTO RIESGO (>40% deserción)
    if x in [39, 7, 42, 2, 26, 27]:
        return 'Alto_Riesgo'
    
    # RIESGO MEDIO (30-40% deserción)
    elif x in [43, 18, 51, 10]:
        return 'Riesgo_Medio'
    
    # BAJO RIESGO (<30% deserción)
    else:  # 1, 17, 44, 15, 16, 5, 53, 57
        return 'Bajo_Riesgo'

df['application_mode_risk'] = df['application_mode'].apply(agrupar_application_mode)

# Verificar
print(df['application_mode_risk'].value_counts())
print("\nTasa de deserción por grupo:")
print(df.groupby('application_mode_risk')['target'].apply(lambda x: (x == 'Dropout').mean() * 100).round(2))


df['application_mode_risk'].hist()


In [ ]:
#### REVISAR ###
"""
HALLAZGO: "Los estudiantes que ingresan por la vía 'Mayor de 23 años' (código 39) presentan una tasa de deserción del 55.4%, casi triple que los estudiantes 
de 1ra fase general (20.2%). Este grupo representa el 17.7% del dataset (785 estudiantes) y constituye uno de los factores de riesgo más importantes identificado. 
Posible explicación: estudiantes adultos con responsabilidades laborales y familiares que dificultan la dedicación al estudio."**
"""
# Variable binaria específica para "Mayor de 23"
df['is_over_23_entry'] = (df['application_mode'] == 39).astype(int) 

print(df['is_over_23_entry'].value_counts())
print("\nTasa de deserción is_over_23_entry:")
print(df.groupby('is_over_23_entry')['target'].apply(lambda x: (x=='Dropout').mean()*100).round(2))




### 2.1.3 Variable course

In [ ]:
### REVISAR ####
# =============================================================================
# 3. COURSE → Target Encoding (No se aplicará aqui, sino en modelado)
# =============================================================================
# NOTA: Se aplica SOLO en train para evitar data leakage
# from category_encoders import TargetEncoder
# te = TargetEncoder(cols=['course'], smoothing=0.3)
# df['course_encoded'] = te.fit_transform(df['course'], y)

### 2.1.4 Variable previous_qualification

In [ ]:
# Se agrupa en 3 categoria por nivel de riesgo
def agrupar_previous_qualification_riesgo(x):
    # ALTO RIESGO: Incompleta + Ed. superior previa
    if x in [9, 10, 14, 15, 2, 3, 19, 12, 5, 4, 6]:
        return 'Alto_Riesgo'
    # RIESGO MEDIO
    elif x in [38, 40]:
        return 'Riesgo_Medio'
    # BAJO RIESGO: Secundaria, técnicos
    else:
        return 'Bajo_Riesgo'

df['previous_qualification_risk'] = df['previous_qualification'].apply(agrupar_previous_qualification_riesgo)
df['previous_qualification_risk'].hist()



### 2.1.5 Variable nationality - SE EXCLUYE DEL MODELO - VARIABLE INTERNATIONAL YA CAPTURA A ESTUDIANTES EXTRANJEROS

### 2.1.6 Variable mothers_qualification y fathers_qualification

In [ ]:
def agrupar_parent_qualification(x):
    if x == 34:
        return 'Desconocido'
    elif x in [35, 36, 20, 13, 25, 33, 31]:
        return 'Sin_Educacion'
    elif x == 37:
        return 'Basica_Baja'
    elif x in [38, 19, 11, 30, 26, 29]:
        return 'Basica_Media'
    elif x in [1, 12, 9, 10, 14, 15, 18, 22, 27]:
        return 'Secundaria'
    elif x in [2, 3, 4, 5, 6, 39, 40, 41, 42, 43, 44]:
        return 'Superior'
    else:
        return 'Desconocido'

df['mothers_qualification_level'] = df['mothers_qualification'].apply(agrupar_parent_qualification)
df['fathers_qualification_level'] = df['fathers_qualification'].apply(agrupar_parent_qualification)

df['mothers_qualification_level'].hist()


In [ ]:
df['fathers_qualification_level'].hist()

### 2.1.7 Variable mothers_occupation y fathers_occupation

In [ ]:
def agrupar_parent_occupation(x):
    if x in [90, 99]:
        return 'Sin_Info'
    elif x == 0:
        return 'Estudiante'
    elif x in [1, 2, 3]:
        return 'Profesional'
    else:
        return 'Otro_Trabajo'

df['mothers_occupation_level'] = df['mothers_occupation'].apply(agrupar_parent_occupation)
df['fathers_occupation_level'] = df['fathers_occupation'].apply(agrupar_parent_occupation)

df['fathers_occupation_level'].hist()

In [ ]:
df['fathers_occupation_level'].hist()

### 2.1.8 Variable aplication_order -- SE MANTIENE

### 2.1.9 FEATURE DERIVADA has_unknown_parent_info

In [ ]:
""" Esta variable se crea por razones: 
1-. Cuando la variable mothers_qualification y/o fathers_qualification tienen código 34 ('Desconocio', es decir, no se tiene información del padre o madre) la tasa de deserción está sobre el 70% er bamos casos.
2-. Cuando la variable mothers_occupation y/o fathers_occupation tienen código 90 ('Otra situación', es decir, no categorizado para el padre o madre), 99 (Descripción en blanco) o 0 (Estudiante) la tasa de deserción 
está entre el 64% y 77% en ambos casos.

"""

df['has_unknown_parent_info'] = (
    (df['mothers_qualification'] == 34) | 
    (df['fathers_qualification'] == 34) |
    (df['mothers_occupation'].isin([90, 99, 0])) |
    (df['fathers_occupation'].isin([90, 99, 0]))
).astype(int)
print(df['has_unknown_parent_info'].value_counts())
print("\nTasa de deserción:")
print(df.groupby('has_unknown_parent_info')['target'].apply(lambda x: (x=='Dropout').mean()*100).round(2))


# 3. TARGET BINARIO

In [ ]:
# =============================================================================
# PASO 1: Crear variable target binaria
# =============================================================================
df['target_binario'] = (df['target'] == 'Dropout').astype(int)
# 1 = Dropout, 0 = No Dropout (Graduate + Enrolled)
df.head() 

# Calculo y gráfica del "desbalance" luego del establecimienot del target binario
plt.figure(figsize=(5, 4))   
df['target_binario'].value_counts(normalize=True).plot(
    kind='bar',
    color=['#2ca02c', '#E74C3C']
)

OUTPUT_DIR = "../outputs/figures/preprocesamiento/"
os.makedirs(OUTPUT_DIR, exist_ok=True)
# Nombre archivo 
filename = f"01_distribucion_target_binario.png"
filepath = os.path.join(OUTPUT_DIR, filename)

plt.title("Distribución de la variable objetivo clase binaria", fontsize=12)
plt.xlabel("Clase", fontsize=10)
plt.ylabel("Proporción", fontsize=10)
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig(filepath, dpi=300, bbox_inches='tight')
plt.show()

df['target_binario'].value_counts()

# 4.0 ELIMINAR VARIABLES 

In [ ]:

# Se eliminan debido a que se creo variable de agrupación
# VARIABLES: marital_status, application_mode, previous_qualification, mothers_qualification, fathers_qualification, mothers_occupation, fathers_occupation

# Se elimina target ya que de deja como binario
# VARIABLE: target

# Se elimina debido a que no es un buen predictor, el 99% de los estidiantes es de Portugal
# VARIABLE: nacionality

variables_a_eliminar = [
    "marital_status",
    "application_mode",
    "previous_qualification",
    "mothers_qualification",
    "fathers_qualification",
    "mothers_occupation",
    "fathers_occupation",
    "nacionality",
    "target"
]

# Eliminar solo si existen en el dataframe
df = df.drop(columns=[c for c in variables_a_eliminar if c in df.columns])

print("Columnas eliminadas del dataset:")
print([c for c in variables_a_eliminar if c not in df.columns])

### REVISAR ###

# Se eliminan debido a que se creo variable de agrupación
# VARIABLES: unemployment_rate, inflation_rate, gdp

In [ ]:
print("Preprocesamiento de variables categóricas completado")

# 5.0 GUARDAR DF PROCESADO

In [ ]:
output_path = "../data/processed/preprocessed_data.csv"
df.to_csv(output_path, index=False)
print(f"Dataset procesado guardado en: {output_path}")

df.head()

